## COVID19 related papers that published during the first week of May, 2020

In [1]:
import pandas as pd
import numpy as np
import os
import json
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import parse_cord as cord

AttributeError: module 'regex' has no attribute 'compile'

In [ ]:
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

In [ ]:
# For Notebooks
init_notebook_mode(connected=True)

# For offline use
cf.go_offline()

### Define a global variable for the folder name that contains the json files of the papers

In [4]:
DATA_DIR = 'data/archive'

Call the module to read in the data

In [5]:
# Use the module that reads in the json files - parse_cord file
data_frame = cord.read_json_files(cord.json_files(DATA_DIR), DATA_DIR)

### Extracting sentences that contain symptoms

Defining the symptoms that we will extract from our sentences. The list was taken from: https://www.kaggle.com/davidbetancur8/symptoms-word-cloud

The list was modified by Max to have more up to date symptoms and I added in the following symptoms which were taken from CDC's website or I observed them frequintly in the papers:

    -"difficulty breathing"
    -"muscle ache"
    -"congestion"
    -"runny nose"
    -"trouble breathing"
    -"persistent pain"
    -"pressure in the chest"
    -"inability to wake"
    -"stay awake"
    -"bluish lips"
    -"bluish face"
    -"fevers"
    -"decreased appetite"

In [6]:
symptoms = [
    "weight loss","chills","shivering","convulsions","deformity","discharge","dizziness", "lymphopenia", "sneezing",
    "vertigo","fatigue","malaise","asthenia","hypothermia","jaundice","muscle weakness", "chest discomfort",
    "pyrexia","sweats","swelling","swollen","painful lymph node","weight gain","arrhythmia", "loss of smell", 
    "loss of appetite", "loss of taste", "bradycardia","chest pain","claudication","palpitations","tachycardia",
    "dry mouth","epistaxis", "dysgeusia", "hypersomnia", "taste loss", "halitosis","hearing loss","nasal discharge", 
    "nasal inflammation", "otalgia","otorrhea","sore throat","toothache","tinnitus", "dysphonia",
    "trismus","abdominal pain","fever","bloating","belching","bleeding","bloody stool","melena","hematochezia", 
    "burning sensation in the chest", "constipation","diarrhea","dysphagia","dyspepsia","fecal incontinence",
    "flatulence", "heartburn", "chest tightness", "chest pressure","nausea","odynophagia","proctalgia fugax",
    "pyrosis","steatorrhea","vomiting","alopecia","hirsutism", "tachypnoea", "nasal obstruction",
    "hypertrichosis","abrasion","anasarca","bleeding into skin","petechia","purpura","ecchymosis", "bruising", 
    "blister","edema","itching","laceration","rash","urticaria","abnormal posturing","acalculia","agnosia","alexia",
    "amnesia","anomia","anosognosia","aphasia","apraxia","ataxia","cataplexy","confusion","dysarthria", 
    "nasal congestion","dysdiadochokinesia","dysgraphia","hallucination","headache","akinesia","bradykinesia",
    "ballismus","blepharospasm","chorea","dystonia","fasciculation","muscle cramps","myoclonus","opsoclonus",
    "tremor","flapping tremor","insomnia","loss of consciousness","syncope","neck stiffness","opisthotonus",
    "paralysis","paresis","paresthesia","prosopagnosia","somnolence","abnormal vaginal bleeding", "neuralgia",
    "vaginal bleeding in early pregnancy", "miscarriage","vaginal bleeding in late pregnancy","amenorrhea", "body aches",
    "infertility","painful intercourse","pelvic pain","vaginal discharge","amaurosis fugax","amaurosis", "skin lesions",
    "blurred vision","double vision","exophthalmos","mydriasis","miosis","nystagmus","amusia","anhedonia",
    "anxiety","apathy","confabulation","depression","delusion","euphoria","homicidal ideation","irritability",
    "mania","paranoid ideation","suicidal ideation","apnea","hypopnea","cough","dyspnea","bradypnea","tachypnea",
    "orthopnea","platypnea","trepopnea","hemoptysis","pleuritic chest pain","sputum production","arthralgia",
    "back pain","sciatica","urologic","dysuria","hematospermia","hematuria","impotence","polyuria",
    "retrograde ejaculation","strangury","urethral discharge","urinary frequency","urinary incontinence", 
    "anosmia", "myalgia", "rhinorrhea", "shortness of breath", "difficulty breathing", "muscle ache", "congestion",
    "runny nose", "trouble breathing", "persistent pain", "pressure in the chest", "inability to wake", "stay awake",
    "bluish lips", "bluish face","akathisia","athetosis", "urinary retention", "fevers", 
    "decreased appetite"]

In [7]:
len(symptoms)

209

## Check the papers that contain any of the symptoms from list of symptoms and then extract those sentences only

In [8]:
#I had to create a df with no title as I could not work withe the frame that includes paper titles as well
df_no_title = data_frame[['paper_id', 'full_text']]

### Final data frame that only includes sentences from each text that contains any of the symptoms from our list of symptoms

    - The split text module takes in a data frame that has paper id and full text. 

    - It Preprocesses the texts by splitting them into one sentence per row and which paper id they belog to
    
    - Then it extractes those sentences that include any of the given sentences and produce a data frame

In [9]:
#Use split text module to preprocess the texts and outputs a data frame of sentences that include one or more of the symptoms
import split_text as sp

final_df = sp.sentence_w_symptoms(sp.split_sentences(df_no_title), symptoms)

If you wish to retrieve sentences that check for different symptoms - then you only need to change the second argument on the split sentence function above

final_df['Sentence'].head()

### Rearranged the display of the columns

In [10]:
columnsTitles = ['Paper_Id', 'Sentence_ID', 'Sentence']

covid_df = final_df.reindex(columns=columnsTitles)

### Output of the test data file 

Save the sentences in a csv file where there are 3 columns: paper ID, sentence ID, and sentence text to be our testing data. This dataframe includes full texts per row and the paper id they belog to.

I did not save the indexes as it would create multiple index columns when we reread in the file.

In [ ]:
covid_df.to_csv('/Users/elif/Desktop/covid_testing_data_May.csv', index = False, encoding= 'utf-8')

## Characteristics of data

Create a program that will take a csv file and report:
 - number of sentences
 - number of sentences with terms
 - individual term counts (how many times each term appears
 - given a collection of symptom terms X in a csv file, for each term in X, its count in the collection, sorted in descending order

## Function that produces total number of sentences, papers that include our symptoms 

In [11]:
def char_of_data(df):
    '''
    functiono produces data characteristics such as number of sentences in the data frame, how many of those have 
    any of the symptoms and how many times each smyptom accour in these sentences. 
    
    :param df: data frame that includes the sentence and paper_id for our data
    returns sorted in descending order of the symptoms and the summary 
    '''
    #print('Total number of papers published in the month of May', len(new_df['Paper_Id'].unique()))
    #print('Total number of sentences from the papers that are published in the given time frame is', len(new_df['Sentence']))
    print('Total number of sentences in the final data frame with symptoms is', len(df['Sentence']))
    print('Total number of unique papers in the final data frame is', len(df['Paper_Id'].unique()))


In [12]:
char_of_data(covid_df)

Total number of sentences in the final data frame with symptoms is 27265
Total number of unique papers in the final data frame is 5970


## Function that produces the count of symptoms in the given data frame

In [13]:
#use df symptom count module
import df_symptom_count as sym_count
sym_df = sym_count.symptoms_df(covid_df, symptoms)

In [14]:
sym_df.head()

,Counts
Symptoms,
fever,7145
anxiety,5130
cough,4775
depression,2990
discharge,2601


Sym_df shows how many time each symptom from our list of symptoms appear ont he given data frame.
If you wish to check the count of different list of symptoms, please update the second parameter in the above function call.
### Visualization of symptom counts

In [15]:
sym_df.iplot(kind='scatter',y='Counts',mode='markers',size=10)

Convert symptoms to dictionary with their counts

In [16]:
sym_df.to_dict()

{'Counts': {'fever': 7145,
  'anxiety': 5130,
  'cough': 4775,
  'depression': 2990,
  'discharge': 2601,
  'diarrhea': 1582,
  'fatigue': 1393,
  'dyspnea': 1358,
  'bleeding': 1207,
  'headache': 1138,
  'edema': 1092,
  'lymphopenia': 1048,
  'shortness of breath': 1026,
  'anosmia': 945,
  'vomiting': 909,
  'confusion': 787,
  'nausea': 733,
  'urologic': 727,
  'myalgia': 721,
  'sore throat': 651,
  'rash': 585,
  'sneezing': 501,
  'abdominal pain': 490,
  'arrhythmia': 457,
  'congestion': 440,
  'chest pain': 409,
  'weight loss': 398,
  'tachycardia': 325,
  'insomnia': 294,
  'dizziness': 245,
  'swelling': 238,
  'dysgeusia': 222,
  'chills': 222,
  'loss of smell': 214,
  'urticaria': 207,
  'malaise': 200,
  'paralysis': 185,
  'dysphagia': 175,
  'rhinorrhea': 174,
  'nasal congestion': 158,
  'ataxia': 154,
  'suicidal ideation': 153,
  'skin lesions': 151,
  'apnea': 144,
  'weight gain': 143,
  'loss of taste': 142,
  'runny nose': 129,
  'bradycardia': 127,
  'nasal

In [17]:
#List of symptoms that appeared in the published papers that were extracted with our list of symptoms
counted_symptoms = sym_df.index.to_list()

### Check to see if any of the symptoms do not appear in the papers from our original symptoms list

In [18]:
main_list = np.setdiff1d(symptoms,counted_symptoms)
# yields the elements in `list_2` that are NOT in `list_1`
len(main_list)

28

It appears that that are 28 symptoms on our original symptoms list that do not occur in the papers

In [19]:
df_test1= covid_df.Sentence.str.extractall('({})'.format('|'.join(main_list)), flags = re.IGNORECASE)\
                           .iloc[:, 0].str.get_dummies().sum(level=0)

### 3. Encountered problem: 'fevers' symptom appears on the list of symptoms that does not occur in the papers. 

I am not able to find out why. It is clear that it does appear over 196 times

In [20]:
print(df_test1.sum(axis=0))

Fevers      4
fevers    192
dtype: int64


###   Create another spreadsheet with three columns: sentence #, word, and tag. 


    Place one word in a row and label symptoms words in the tag column: 
    mark the beginning (B-Sym) and inside (I-Sym) of each symptom term. 
    If a term consists of only one word, simply mark it as B-Sym with no I-Sym.  
    Label all other words as O. 

In [25]:
def repl(m):
    '''
    regular expression function that replaces symptoms with BSYM if .group(0) is one word
    or BSYM + ISYM if multiple words - this function looks for spaces in the given words

    m.group(0) - firts match
    '''
    return ' '.join(['BSYM']+['ISYM']*(m.group(0).count(' ')))

In [28]:
pattern = '\\b' + '\\b|\\b'.join(sym) + '\\b'
covid_df['Token'] = covid_df['Sentence'].str.replace(pattern, repl, flags = re.IGNORECASE)

NameError: name 'sym' is not defined

In [22]:
covid_df.reset_index(drop=True, inplace=True)

In [23]:
covid_df['Sentence_ID'] = ['Sentence #%s' %i for i in range(1, len(covid_df) + 1)]

### Tokenize sentences

In this part of the task, I tokenized words and tokens columns and then replaces the tokens
with the actual display of B-SYM and I-SYM with the dashes added in the middle.

Then, provided a dataframe with three columns only

In [24]:
#Use symptom tagging module to tag the words based on our symptom list
import symptom_tagging as sym_tag
tagged_data = sym_tag.remove_duplicate_sentence_ids(sym_tag.tokenize_sentences(covid_df))


KeyError: 'Token'

Double checking the tag column to see if we were able to tag I-SYM symptom words

In [ ]:
tag_df[tag_df['Tag']== 'I-SYM'].head(5)

In [46]:
###  Here the sentence # should be consecutive integers (similar to a surrogate key in a database table) and is not the same as sentence ID in the first spreadsheet.

    Styling of sentence numbers ~ instead of duplicating the same number over and over again, we leave out the duplicated ones by replacing them with space

,Sentence_ID,Words,Tag
181,Sentence #3,pain,I-SYM
213,Sentence #4,pain,I-SYM
223,Sentence #4,appetite,I-SYM
271,Sentence #5,pain,I-SYM
285,Sentence #6,chest,I-SYM


#tag_df['Sentence_ID'] = tag_df['Sentence']

#Final tagged covid data
tagged_data.to_csv('/Users/elif/Desktop/covid_tagged_data_May.csv', index = False, encoding= 'utf-8')


In [55]:
#Sort the dictionary of symptoms along with their value counts
import operator
sorted_d = dict( sorted(new_dict.items(), key=operator.itemgetter(1),reverse=True))
# sorted_d

In [56]:
df_sym_freq = pd.DataFrame(list((dict(new_dict)).items()), columns = ['Symptom', 'Frequency'])
df_sym_freq.to_csv('sym_freq.csv',index=False)
only_syms = list((dict(new_dict)).keys())

In [58]:
#Wordcloud of symptoms 

# wc = WordCloud(background_color = "black", width = 1000, height= 1000).generate_from_frequencies(new_dict)
# fig = plt.figure(figsize = (15,15))
# plt.imshow(wc, interpolation = "bilinear")
# plt.axis("off")
# plt.show()

In [59]:
df_sym_freq.groupby('Symptom').sum().sort_values(['Frequency'],ascending=False).head()

In [60]:
# only_syms
print('Total number of symptoms retrived from both data sets is:', len(only_syms))

In [61]:
# yields the elements in `list_2` that are NOT in `list_1`
final_list = np.setdiff1d(symptoms,only_syms)
print('Total number of symptoms that are retrived from both data sets that do not exist in the original symptom list is:', len(final_list))

,Frequency
Symptom,
cough,41
fever,32
sore throat,16
fatigue,15
headache,15


In [62]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

Total number of symptoms retrived from both data sets is: 400


In [63]:
print('Total number of symptoms that appear in both the original list and the list created from colloquial data sets is:', len(intersection(symptoms, only_syms)))

Total number of symptoms that are retrived from both data sets that do not exist in the original symptom list is: 177


In [64]:
### Create a version of the COLL-DATA that only contains *sentences* with symptom terms

In [65]:
import colloq_tagged_data_processing as coll_process

sent_value, colloqual_df = coll_process.colloqual_data_processing(combined_colloquial_df, )

Total number of symptoms that appear in both the original list and the list created from colloquial data sets is: 32


In [92]:
def colloqual_df_info(sent, df):
    print('Total number of sentences in this file is:', len(sent))
    print('Number of sentences that include any of the provided symptoms is:', len(df))
    print('Number of sentences that do NOT include any of the provided symptoms is:', (len(sent) - len(df)))
    
    

,Sentence,Words,Tag
0,Sentence #1,I,O
1,,was,O
2,,diagnosed,O
3,,3,O
4,,days,O


In [67]:
sent_value, colloqual_df = colloqual_data_processing(plm_df, only_syms)

In [68]:
def colloqual_df_info(sent, df):
    print('Total number of sentences in this file is:', len(sent))
    print('Number of sentences that include any of the provided symtoms is:', len(df))
    print('Number of sentences that do NOT include any of the provided symptoms is:', (len(sent) - len(df)))
    
    

In [69]:
colloqual_df_info(sent_value, colloqual_df)

Total number of sentences in this file is: 531
Number of sentences that include any of the provided symtoms is: 168
Number of sentences that do NOT include any of the provided symptoms is: 363


In [70]:
sent_value_, colloqual_df_n = colloqual_data_processing(paSi_df, only_syms)

In [71]:
colloqual_df_info(sent_value_, colloqual_df_n)

Total number of sentences in this file is: 853
Number of sentences that include any of the provided symtoms is: 301
Number of sentences that do NOT include any of the provided symptoms is: 552


## Put the colloquial data prior to Nov5 sentences into tagged format

In [72]:
colloqual_df.reset_index(drop=True, inplace=True)

In [73]:
plm_peS_df = sentence_w_symptoms(colloqual_df, symptoms)

In [74]:
plm_peS_df['Token'] = plm_peS_df['Sentence'].str.replace(pattern, repl, flags = re.IGNORECASE)
plm_peS_df['Sentence_ID'] = ['Sentence #%s' %i for i in range(1, len(plm_peS_df) + 1)]


In [75]:
df5 = tokenize_sentences(plm_peS_df)
plm_peS_df_out=remove_duplicate_sentence_ids(df5)

In [76]:
plm_peS_df_out=remove_duplicate_sentence_ids(df5)

In [77]:
plm_peS_df_out.to_csv('/Users/elif/Desktop/Existing_Colloquial_data_prior_Nov5.csv', index = False, encoding= 'utf-8')

## Scraped Data from the sites

    Created one csv file where all the text data fromo both sites is stored
    Loaded the csv file to select the sentences that include symptoms
    Output the tagged file for symptoms 

In [78]:
scraped_data = pd.read_csv('/Users/elif/Downloads/combined_csv-4.csv')

In [79]:
#Count of original symptoms
symptoms_df(scraped_data)#, symp = symptoms)

,Counts
Symptoms,
cough,61
fever,49
headache,33
fatigue,29
anxiety,28
sore throat,17
shortness of breath,16
dizziness,12
vertigo,10


In [80]:
len(symptoms_df(scraped_data))

43

In [81]:
#count of symptomps from the patience like me files
symptoms_df(scraped_data, only_syms)

,Counts
Symptoms,
symptoms,146
feeling,63
breath,49
cough,46
taste,43
...,...
hives,1
high temperatures,1
hayfever,1


In [89]:
#Make sure there is only one sentence per row
sentences = []
sentence_pattern = r'(?<=[^A-Z].[.?]) +(?=[A-Z])'
for row in scraped_data.itertuples():            
    for sentence in re.split(sentence_pattern, row[1]):
        sentences.append((row[0], sentence))
                

collo_df_new = pd.DataFrame(sentences, columns=['Index','Sentence'])

collo_df_new.drop('Index', axis = 1, inplace = True)
print('Number of sentences retrieved from the sites for the past 3 months is: ', len(collo_df_new))

Number of sentences retrived from the sites for the past 3 months is:  1292


In [91]:
colloqual_new_data = sentence_w_symptoms(collo_df_new, symptoms)
print('Of those sentences ', len(colloqual_new_data), 'of them include our original Symptoms')

Of those sentences  203 of them include our original Symptoms


In [90]:
colloqual_new_data_only_symp = sentence_w_symptoms(collo_df_new, only_syms)
print('Of those sentences ', len(colloqual_new_data_only_symp), 'of them include Symptoms collected from previous colloquial datasets.')

Of those sentences  471 of them include Symptoms collected from previous colloquial datasets.


In [84]:
colloqual_new_data['Token'] = colloqual_new_data['Sentence'].str.replace(pattern, repl, flags = re.IGNORECASE)
colloqual_new_data['Sentence_ID'] = ['Sentence #%s' %i for i in range(1, len(colloqual_new_data) + 1)]
df6 = tokenize_sentences(colloqual_new_data)
tagged_scraped_data = remove_duplicate_sentence_ids(df6)

In [85]:
tagged_scraped_data.to_csv('/Users/elif/Desktop/New_Colloquial_data_Nov5.csv', index = False, encoding= 'utf-8')

## Things to address - Nov 6 Meeting

    - Changed the way we split sentences into one sentence per row format, thus, we now have a different testing set for the MAY papers which is saved in MAX&ELIF folder
    
    - The existing tagged colloquial data sets were split in a different fashion thus the quality of it might be different that the other files - such as the split of I've or numbers split in different cells
    
    - There were not many posts on those 2 sites that were covid related as they had many posts about just updates on recent changes in the vaccine process
    
    - Scraped data was saved in different csv files and we merged them into one csv file to capture all the information 
    
    - Some of the posts on the site had the edit date as - 2 weeks or months - but some of the comments dated back to 4-5 months ago so not sure about what part of that post is actually recently edited
    
    - The sites do not have acrtual time stamps on the posts thus, I went back 3 months to collect all the information that we have right now.